# 🚀 کسان مددگار - Plant Disease Detection (Google Colab)

## 📋 Instructions:
1. **Connect to Colab:** Select Kernel → Google Colab → T4 GPU
2. **Run cells one by one** (Cell 1 → Cell 10)
3. **Download model** when training completes

## 🎯 This Notebook:
- **Dataset:** PlantVillage COLOR folder only (38 classes)
- **Model:** EfficientNet-B4 
- **Target Accuracy:** 85-95%
- **Auto-save:** Google Drive (prevents loss on disconnect)

## ⚠️ Previous Issue Fixed:
- OLD: 152 classes (color + grayscale + segmented = duplicates)
- NEW: 38 unique classes (COLOR only) → No overfitting!

Let's begin! 👇

In [6]:
# ============================================
# CELL 1: Setup Kaggle API (VS Code + Colab)
# ============================================
import os
from pathlib import Path
import json

print("="*70)
print("🔧 SETTING UP KAGGLE API (VS Code + Colab)")
print("="*70)

# Kaggle credentials (from kaggle.json in repo)
kaggle_credentials = {
    "username": "Ammmaaarrrr",
    "key": "KGAT_448e734a3925f9b1cfc82a38e22445a0"
}

# Setup kaggle directory
kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)
kaggle_json = kaggle_dir / "kaggle.json"

# Write credentials
with open(kaggle_json, 'w') as f:
    json.dump(kaggle_credentials, f)

try:
    os.chmod(kaggle_json, 0o600)
except:
    pass  # Windows may not support chmod

print(f"✅ Kaggle API configured at: {kaggle_json}")

# Set environment variables (alternative method)
os.environ['KAGGLE_USERNAME'] = kaggle_credentials['username']
os.environ['KAGGLE_KEY'] = kaggle_credentials['key']

# Install kaggle CLI
print("\n📦 Installing kaggle CLI...")
import subprocess
result = subprocess.run(["pip", "install", "-q", "kaggle"], capture_output=True)

print("\n" + "="*70)
print("✅ Kaggle API setup complete!")
print("="*70)


: 

In [7]:
# ============================================
# CELL 2: Mount Drive & Download Dataset (COLOR ONLY)
# ============================================
import os
import subprocess
import shutil
from pathlib import Path

print("="*70)
print("📂 DOWNLOADING PLANTVILLAGE DATASET (COLOR ONLY)")
print("="*70)

# Mount Google Drive for auto-save
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive mounted")
    DRIVE_SAVE = Path("/content/drive/MyDrive/trained_models")
    DRIVE_SAVE.mkdir(parents=True, exist_ok=True)
except:
    print("⚠️ Drive mount skipped")
    DRIVE_SAVE = Path("/content/saved_models")
    DRIVE_SAVE.mkdir(parents=True, exist_ok=True)

# Download PlantVillage dataset
print("\n📦 Downloading PlantVillage dataset...")
os.system("kaggle datasets download -d abdallahalidev/plantvillage-dataset -p /content/data/ --unzip")

# ⚠️ USE ONLY COLOR FOLDER (38 unique classes)
COLOR_DIR = Path("/content/data/plantvillage dataset/color")
DATA_DIR = Path("/content/data/plantvillage_color")

if COLOR_DIR.exists():
    # Copy only color folder
    if DATA_DIR.exists():
        shutil.rmtree(DATA_DIR)
    shutil.copytree(COLOR_DIR, DATA_DIR)
    print(f"✅ Using COLOR folder only: {DATA_DIR}")
else:
    # Try alternate path
    alt_paths = [
        Path("/content/data/PlantVillage/color"),
        Path("/content/data/plantvillage/color"),
    ]
    for alt in alt_paths:
        if alt.exists():
            if DATA_DIR.exists():
                shutil.rmtree(DATA_DIR)
            shutil.copytree(alt, DATA_DIR)
            print(f"✅ Using COLOR folder: {alt}")
            break
    else:
        DATA_DIR = Path("/content/data")
        print("⚠️ Color folder not found, using all data")

# Count classes
class_folders = [f for f in DATA_DIR.iterdir() if f.is_dir()]
total_images = 0

print(f"\n📊 Found {len(class_folders)} classes:")
for folder in sorted(class_folders)[:10]:
    count = len(list(folder.glob('*')))
    total_images += count
    print(f"   📂 {folder.name}: {count:,} images")

if len(class_folders) > 10:
    for folder in sorted(class_folders)[10:]:
        total_images += len(list(folder.glob('*')))
    print(f"   ... and {len(class_folders) - 10} more classes")

print(f"\n✅ Total: {len(class_folders)} classes, {total_images:,} images")
print(f"💾 Auto-save path: {DRIVE_SAVE}")

: 

In [ ]:
# ============================================
# CELL 3: Install & Import Libraries
# ============================================
!pip install -q timm

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json
from tqdm import tqdm
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import timm

# Speed optimizations
torch.backends.cudnn.benchmark = True
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Libraries imported!")
print(f"🖥️ Device: {device}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

🔧 SETTING UP KAGGLE API (VS Code + Colab)
✅ Kaggle API configured at: /root/.kaggle/kaggle.json

📦 Installing kaggle CLI...

✅ Kaggle API setup complete!

📥 Downloading 'rice' dataset...
❌ Failed to download 'rice': 
⚠️ Check if Kaggle API is configured and dataset exists (404 if wrong).

📥 Downloading 'cotton' dataset...
❌ Failed to download 'cotton': 
⚠️ Check if Kaggle API is configured and dataset exists (404 if wrong).

📥 Downloading 'mango' dataset...
❌ Failed to download 'mango': 
⚠️ Check if Kaggle API is configured and dataset exists (404 if wrong).

📥 Downloading 'wheat' dataset...
❌ Failed to download 'wheat': 
⚠️ Check if Kaggle API is configured and dataset exists (404 if wrong).

📂 Data folders in /content/data:
   - plantvillage dataset


In [ ]:
# ============================================
# CELL 4: Load Dataset (38 Classes)
# ============================================
print("="*70)
print("📊 LOADING DATASET")
print("="*70)

# Find all classes
all_classes = {}
image_extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}

for class_folder in DATA_DIR.iterdir():
    if class_folder.is_dir():
        images = [str(f) for f in class_folder.iterdir() if f.suffix in image_extensions]
        if len(images) > 0:
            all_classes[class_folder.name] = images

print(f"✅ Found {len(all_classes)} classes")

# Build dataset
all_image_paths = []
all_labels = []
class_names = []

for idx, (class_name, paths) in enumerate(sorted(all_classes.items())):
    class_names.append(class_name)
    all_image_paths.extend(paths)
    all_labels.extend([idx] * len(paths))

num_classes = len(class_names)
print(f"✅ Total: {len(all_image_paths):,} images, {num_classes} classes")

# Show sample classes
print(f"\n📋 Classes:")
for i, name in enumerate(class_names):
    count = sum(1 for l in all_labels if l == i)
    print(f"   {i+1:2d}. {name}: {count:,} images")

✅ Setup complete!
🖥️  Device: cuda
🎮 GPU: Tesla T4
💾 GPU Memory: 15.8 GB


In [ ]:
# ============================================
# CELL 5: Create Dataset Class & Transforms
# ============================================
print("="*70)
print("🔧 CREATING DATASET")
print("="*70)

class PlantDiseaseDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        try:
            image = Image.open(self.image_paths[idx]).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, self.labels[idx]
        except:
            image = Image.new('RGB', (224, 224), (0, 0, 0))
            if self.transform:
                image = self.transform(image)
            return image, self.labels[idx]

# Strong augmentation to prevent overfitting
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2)
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("✅ Dataset class and transforms created")
print("🔄 Using strong augmentation to prevent overfitting")

📊 ANALYZING DATASETS

🔍 Analyzing color...
   Found: 38 classes, 54,305 images

🔍 Analyzing grayscale...
   Found: 38 classes, 54,305 images

🔍 Analyzing segmented...
   Found: 38 classes, 54,306 images

📈 TOTAL: 114 classes, 162,916 images


In [ ]:
# ============================================
# CELL 6: Create Data Loaders
# ============================================
print("="*70)
print("📊 CREATING DATA LOADERS")
print("="*70)

# Create dataset
full_dataset = PlantDiseaseDataset(all_image_paths, all_labels, train_transform)

# Split: 70% train, 15% val, 15% test
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Update val/test transforms (no augmentation)
val_ds.dataset = PlantDiseaseDataset(all_image_paths, all_labels, test_transform)
test_ds.dataset = PlantDiseaseDataset(all_image_paths, all_labels, test_transform)

# Data loaders
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, 
                          num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, 
                        num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, 
                         num_workers=2, pin_memory=True)

print(f"\n📊 Dataset Split:")
print(f"   Training:   {len(train_ds):,} images ({len(train_loader)} batches)")
print(f"   Validation: {len(val_ds):,} images ({len(val_loader)} batches)")
print(f"   Test:       {len(test_ds):,} images ({len(test_loader)} batches)")
print(f"\n⚡ Batch size: {batch_size}")

✅ Created dataset: 114 classes, 162,916 images

📋 Sample classes:
   1. color___Corn_(maize)___Common_rust_: 1192 images
   2. color___Orange___Haunglongbing_(Citrus_greening): 5507 images
   3. color___Pepper,_bell___Bacterial_spot: 997 images
   4. color___Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 1076 images
   5. color___Corn_(maize)___healthy: 1162 images
   ... and 109 more classes


In [ ]:
# ============================================
# CELL 7: Create Model (EfficientNet-B4 with Dropout)
# ============================================
print("="*70)
print("🤖 CREATING MODEL")
print("="*70)

# EfficientNet-B4 with dropout for regularization
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=num_classes, drop_rate=0.3)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"🤖 Model: EfficientNet-B4")
print(f"📊 Classes: {num_classes}")
print(f"⚙️ Parameters: {total_params:,}")
print(f"🎯 Dropout: 0.3 (prevents overfitting)")

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label smoothing helps!
optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=0.05)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

print(f"\n⚙️ Optimizer: AdamW (lr=0.0003, weight_decay=0.05)")
print(f"📅 Scheduler: CosineAnnealingWarmRestarts")
print(f"🏷️ Label Smoothing: 0.1")


📊 Dataset Split:
  Training:   114,041 images
  Validation: 24,437 images
  Test:       24,438 images

⚡ Optimizations: batch=64, workers=4, pin_memory=True


In [ ]:
# ============================================
# CELL 8: Training Loop (15 Epochs + Early Stopping)
# ============================================
import time
import shutil

print("="*70)
print("🚀 STARTING TRAINING (15 Epochs)")
print("="*70)

epochs = 15
best_val_acc = 0.0
patience = 5
patience_counter = 0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print(f"💾 Auto-save to: {DRIVE_SAVE}")
print(f"⏰ Early stopping patience: {patience}")

for epoch in range(epochs):
    start = time.time()
    
    # Training
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    for images, labels in pbar:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({'loss': f'{loss.item():.3f}', 'acc': f'{100.*train_correct/train_total:.1f}%'})
    
    train_acc = 100. * train_correct / train_total
    train_loss = train_loss / len(train_loader)
    
    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_acc = 100. * val_correct / val_total
    val_loss = val_loss / len(val_loader)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'class_names': class_names,
            'num_classes': num_classes,
            'history': history
        }
        torch.save(checkpoint, 'pakistan_model_best.pth')
        # Auto-save to Drive
        try:
            shutil.copy('pakistan_model_best.pth', DRIVE_SAVE / 'pakistan_model_best.pth')
            print(f"   💾 Saved to Drive! (Val: {val_acc:.2f}%)")
        except:
            print(f"   💾 Saved locally! (Val: {val_acc:.2f}%)")
    else:
        patience_counter += 1
    
    # Save class_names and history every epoch
    with open('class_names.json', 'w') as f:
        json.dump(class_names, f, indent=2)
    with open('training_history.json', 'w') as f:
        json.dump(history, f, indent=2)
    try:
        shutil.copy('class_names.json', DRIVE_SAVE / 'class_names.json')
        shutil.copy('training_history.json', DRIVE_SAVE / 'training_history.json')
    except:
        pass
    
    scheduler.step()
    elapsed = time.time() - start
    
    print(f"\n📊 Epoch {epoch+1}: Train={train_acc:.1f}%, Val={val_acc:.1f}%, Best={best_val_acc:.1f}%")
    print(f"   ⏱️ {elapsed:.0f}s | LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Early stopping
    if patience_counter >= patience:
        print(f"\n⏹️ Early stopping! No improvement for {patience} epochs")
        break
    print("-"*70)

print(f"\n✅ Training complete! Best: {best_val_acc:.2f}%")

🚀 STARTING TRAINING (Mixed Precision + cuDNN)


Epoch 1/25: 100%|██████████| 1782/1782 [13:49<00:00,  2.15it/s, loss=0.1616, acc=84.4%]



📊 Epoch 1: Train Loss=0.5969, Train Acc=84.43%
   Val Loss=0.1155, Val Acc=95.94%, Best=95.94%
   ⏱️ Time: 931s | ETA: 372.6 min
----------------------------------------------------------------------


Epoch 2/25: 100%|██████████| 1782/1782 [11:59<00:00,  2.48it/s, loss=0.1702, acc=96.6%]



📊 Epoch 2: Train Loss=0.0930, Train Acc=96.59%
   Val Loss=0.0804, Val Acc=97.12%, Best=97.12%
   ⏱️ Time: 807s | ETA: 309.4 min
----------------------------------------------------------------------


...


📊 Epoch 8: Train Loss=0.0175, Train Acc=99.32%
   Val Loss=0.0456, Val Acc=98.50%, Best=98.50%
   ⏱️ Time: 800s | ETA: 226.7 min
----------------------------------------------------------------------


Epoch 9/25: 100%|██████████| 1782/1782 [11:57<00:00,  2.48it/s, loss=0.0091, acc=99.4%]



📊 Epoch 9: Train Loss=0.0148, Train Acc=99.40%
   Val Loss=0.0434, Val Acc=98.49%, Best=98.50%
   ⏱️ Time: 796s | ETA: 212.4 min
----------------------------------------------------------------------


Epoch 10/25: 100%|██████████| 1782/1782 [11:57<00:00,  2.48it/s, loss=0.0060, acc=99.5%]


: 

In [ ]:
# ============================================
# CELL 9: Test & Plot Results
# ============================================
print("="*70)
print("🧪 TESTING & PLOTTING")
print("="*70)

# Load best model
checkpoint = torch.load('pakistan_model_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Test
test_correct, test_total = 0, 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

test_acc = 100. * test_correct / test_total
print(f"\n🎯 Test Accuracy: {test_acc:.2f}%")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history['train_loss']) + 1)

axes[0].plot(epochs_range, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
axes[0].plot(epochs_range, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(epochs_range, history['train_acc'], 'b-', label='Train Acc', linewidth=2)
axes[1].plot(epochs_range, history['val_acc'], 'r-', label='Val Acc', linewidth=2)
axes[1].axhline(y=test_acc, color='g', linestyle='--', label=f'Test: {test_acc:.1f}%')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()

print(f"\n📊 Results saved!")

🚨 RECOVERY: Saving partial training progress

📊 Epochs completed: 0
❌ No training history found. Training may not have started.
   Run Cell 8 to start training first.

🔄 To RESUME training from checkpoint, run Cell 8.6 below


In [ ]:
# ============================================
# CELL 10: Download Model Files
# ============================================
print("="*70)
print("📥 DOWNLOADING MODEL FILES")
print("="*70)

# Save final info
model_info = {
    'class_names': class_names,
    'num_classes': num_classes,
    'test_accuracy': test_acc,
    'best_val_accuracy': best_val_acc,
    'model_architecture': 'efficientnet_b4',
    'total_images': len(all_image_paths),
    'epochs_trained': len(history['train_loss'])
}

with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

# Copy to Drive
try:
    shutil.copy('pakistan_model_best.pth', DRIVE_SAVE / 'pakistan_model_best.pth')
    shutil.copy('class_names.json', DRIVE_SAVE / 'class_names.json')
    shutil.copy('model_info.json', DRIVE_SAVE / 'model_info.json')
    shutil.copy('training_history.json', DRIVE_SAVE / 'training_history.json')
    shutil.copy('training_history.png', DRIVE_SAVE / 'training_history.png')
    print(f"✅ All files saved to Google Drive: {DRIVE_SAVE}")
except Exception as e:
    print(f"⚠️ Drive save failed: {e}")

# Download files
print("\n📥 Downloading files...")
from google.colab import files

files.download('pakistan_model_best.pth')
files.download('class_names.json')
files.download('model_info.json')
files.download('training_history.json')
files.download('training_history.png')

print("\n" + "="*70)
print("🎉 DONE!")
print("="*70)
print(f"\n📊 Final Results:")
print(f"   🏆 Test Accuracy: {test_acc:.2f}%")
print(f"   🏆 Best Val Accuracy: {best_val_acc:.2f}%")
print(f"   📁 Classes: {num_classes}")
print(f"\n📁 Files downloaded - copy to 'Flask Deployed App/' folder")

🔄 RESUMING TRAINING FROM CHECKPOINT
❌ No checkpoint found. Starting from scratch.

🚀 Continuing training for 25 more epochs...


NameError: name 'time' is not defined

## ✅ Training Complete!

### 📁 Downloaded Files:
- `pakistan_model_best.pth` - Model weights
- `class_names.json` - 38 class labels
- `model_info.json` - Metadata
- `training_history.json` - Training log
- `training_history.png` - Training plot

### 🚀 Next Steps:
1. Copy files to `Flask Deployed App/` folder
2. Run Flask app: `python app.py`
3. Test with plant images!

### 🇵🇰 کسان مددگار - Helping Pakistani Farmers!